<br>
<h1 style = "font-size:30px; font-weight : bold; color : black; text-align: center; border-radius: 10px 15px;"> World Development Indicators: Brazil Unemployment - Using SQL for Analysis </h1>
<br>

# Goals
The primary and personal goal that motivated the creation of this notebook was to practice SQL queries. Each specific goal was defined after its previous steps.

Goals:
- Explore the database to understand what type of data is presented in each table;
- Choosing a topic of interest (choice: unemployment) and select some relevant indicators among those available for Brazil;
- Extract the data related to the chosen indicators and prepare them for analysis;
- Create plots to analyze the unemployment in Brazil over time and compare the unemployment rate in different categories (by gender and education level or on the young population).

# <a id='0'>Content</a>

- <a href='#1'>Dataset Information</a>  
- <a href='#2'>Importing Packages and Exploring the Database</a>  
- <a href='#3'>Choosing Indicators</a>  
- <a href='#4'>Extracting and Manipulating the Data for Analysis</a>
- <a href='#5'>References</a>

# <a id="1">Dataset Information</a>

The World Development Indicators from the World Bank contain over a thousand annual indicators of economic development from hundreds of countries around the world.

|Table|Total Rows|Total Columns|
--- | --- | ---
|Country|247|31|
|CountryNotes|4857|3|
|FootNotes|532415|4|
|Indicators|5656458|6|
|Series|1345|20|
|SeriesNotes|369|3|

Last updated (2017)

## <center> If you find this notebook useful, support with an upvote! <center>

# <a id="2">Importing Packages and Exploring the Database</a> 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import sqlite3

Let's start by connecting to the database and creating a cursor to fetch the data

In [ ]:
database_ref = "../input/world-development-indicators/database.sqlite"

conn = sqlite3.connect(database_ref)

cur = conn.cursor()

The first query we will be doing is to get the name of the tables in the database. 

> “Every SQLite database contains a single "schema table" that stores the schema for that database. The schema for a database is a description of all of the other tables, indexes, triggers, and views that are contained within the database” [(SQLite documentation)](https://www.sqlite.org/schematab.html).

We can refer to that schema table as ‘sqlite_master’.


In [ ]:
#Getting tables names by querying 'sqlite_master'

list_of_tables_query = """
                      SELECT name
                      FROM sqlite_master
                      WHERE type='table';
                      """ 
cur.execute (list_of_tables_query)
print(cur.fetchall())

There are 6 tables in the database. Let’s execute some simple queries to see what’s the content in each table. We can use ‘pd.read_sql’ to frame the results as Pandas Dataframes. To better understand what we have, let's take look at each table.

In [ ]:
country_query = """
                SELECT *
                FROM Country
                LIMIT 10;
                """ 
country = pd.read_sql(country_query, conn)

#Show all columns from dataframes #Increase column width
pd.options.display.max_columns = None
pd.options.display.max_colwidth = 100

country.head()

In [ ]:
countrynotes_query = """
                    SELECT *
                    FROM CountryNotes
                    LIMIT 10;
                    """ 
countrynotes = pd.read_sql(countrynotes_query, conn)

countrynotes.head()

In [ ]:
series_query =  """
                SELECT *
                FROM series
                LIMIT 10;
                """
series = pd.read_sql(series_query, conn)

series.head()

In [ ]:
seriesnotes_query = """
                    SELECT *
                    FROM SeriesNotes
                    LIMIT 10;
                    """
seriesnotes = pd.read_sql(seriesnotes_query, conn)

seriesnotes.head()

In [ ]:
indicators_query = """
                    SELECT *
                    FROM Indicators
                    LIMIT 10;
                    """
indicators = pd.read_sql(indicators_query, conn)

indicators.head()

In [ ]:
footnotes_query = """
                    SELECT *
                    FROM FootNotes
                    LIMIT 10;
                    """
footnotes = pd.read_sql(footnotes_query, conn)

footnotes.head()

Few points about some tables:
- Country: Presents basic information about each country. We can use ‘CountryCode’ to execute JOIN with most tables.
- Series: Contains information about the available indicators. Connect with others through 'SeriesCode'.
- Indicators: Where all indicators’ values are stored. Connects with Series using IndicatorCode = SeriesCode and with Country using ‘CountryCode’.

The country of choice to be analyzed is Brazil. Before we move on, let’s just check if Brazil is presented in this database.

In [ ]:
brazil_query = """
                SELECT *
                FROM country
                WHERE ShortName = 'Brazil';
                """

brazil_info = pd.read_sql(brazil_query, conn)
brazil_info

# <a id="3">Choosing Indicators</a>

Let’s see what indicators about Brazil are available. We can search for them using CountryCode = ‘BRA’.  Since for every indicator there are several rows representing the obtained value of each year, we will use SELECT DISTINCT to avoid getting repeated names.

In [ ]:
#Using SELECT DISTINCT to not get repeated values 
brazil_indicators_query = """
                            SELECT DISTINCT IndicatorName
                            FROM Indicators
                            WHERE CountryCode = 'BRA';
                            """

brazil_indicators = pd.read_sql(brazil_indicators_query, conn)
brazil_indicators

There are over a thousand of indicators. Instead of trying to read one by one, it makes much more sense to see what topics are available, choose a topic of interest and then query its indicators.

In [ ]:
topics_query = """
                SELECT DISTINCT Topic
                FROM Series;
                """

topics = pd.read_sql(topics_query, conn)
topics

Putting all topics on a list

In [ ]:
topics_list = [x for x in topics['Topic']] 
topics_list

There is a lot of interesting topics for us to choose. In this notebook, we will be focusing on the ‘unemployment’ topic. Let’s find out which indicators are available.

In [ ]:
indicators_unemployment_query = """
                                SELECT SeriesCode, IndicatorName
                                FROM Series
                                WHERE Topic LIKE '%Unemployment';
                                """
indicators_unemployment = pd.read_sql(indicators_unemployment_query, conn)
indicators_unemployment

There are 27 indicators related to unemployment, but we didn’t specify in the query that the indicators should be available for Brazil. The Series table doesn’t have a column for Country Code, but the Indicators table does. With INNER JOIN, we can define what indicators we want based on conditions related to both tables (Country Code from Indicators and Topic from Series).

In [ ]:
indicators_unemployment_bra_query = """
                                SELECT DISTINCT s.SeriesCode, s.IndicatorName
                                FROM Series AS s
                                INNER JOIN Indicators AS i
                                    ON i.IndicatorCode = s.SeriesCode
                                WHERE i.CountryCode = 'BRA' AND s.Topic LIKE '%Unemployment';
                                """
indicators_unemployment_bra = pd.read_sql(indicators_unemployment_bra_query, conn)
indicators_unemployment_bra

When we define ‘CountryCode’ = ‘BRA’, we find 25 indicators instead of 27. Let’s check which ones aren’t available.

In [ ]:
indicators_unemployment['IndicatorName'].isin(indicators_unemployment_bra['IndicatorName'])

The missing indicators are the ones for long-term unemployment based on gender.

Now that we have the list of available indicators about unemployment, we could simply make a query for each indicator based on its code and plot the result, like in the example below.

In [ ]:
unemployment_bra_query = """
                        SELECT Year, Value
                        FROM Indicators
                        WHERE CountryCode = 'BRA' AND IndicatorCode = 'SL.UEM.TOTL.NE.ZS';
                        """
unemployment_bra = pd.read_sql(unemployment_bra_query, conn)
unemployment_bra

In [ ]:
sns.set_style("darkgrid")

plt.figure(figsize=(14,5))

sns.lineplot(data = unemployment_bra, x = 'Year', y = 'Value', marker = 'o')

plt.title('Unemployment (national estimate)')
plt.ylabel('% of unemployment')
#plt.xticks(unemployment_bra['Year'])

plt.show()

In this notebook, we will extract the information about the desired indicators at once and then treat them with our knowledge on the Pandas package.

We still have some choices to make. We have some indicators that seem to be repeated (e.g. ‘Unemployment, total’) where the difference is how they were obtained (national estimate or modeled ILO estimate). Let’s get the ‘Unemployment, total’ indicators and see how they are different from each other.

In [ ]:
total_unemployment_ilo_query = """
                                SELECT Year, Value
                                FROM Indicators
                                WHERE CountryCode = 'BRA' AND IndicatorCode = 'SL.UEM.TOTL.ZS';
                                """
total_unemployment_ilo = pd.read_sql(total_unemployment_ilo_query, conn)
total_unemployment_ilo

In [ ]:
total_unemployment_ne_query = """
                                SELECT Year, Value
                                FROM Indicators
                                WHERE CountryCode = 'BRA' AND IndicatorCode = 'SL.UEM.TOTL.NE.ZS';
                                """
total_unemployment_ne = pd.read_sql(total_unemployment_ne_query, conn)
total_unemployment_ne

In [ ]:
plt.figure(figsize=(14,5))

sns.lineplot(data = total_unemployment_ilo, x = 'Year', y = 'Value', marker = 'o', label = 'ILO Estimate')
sns.lineplot(data = total_unemployment_ne, x = 'Year', y = 'Value', marker = 'o', label = 'Nat. Estimate')

plt.title('Unemployment in Brazil (ILO vs National Estimates)')
plt.ylabel('% of total unemployment')
plt.legend(loc = 'upper left')

plt.show()

Apart from the fact that the ILO estimate doesn’t have any record before 1991 and that the national estimate has a few missing values, the results between the two are quite similar. I’ll be making the choice of dropping ‘national estimate’ indicators when the ILO estimate is available. If you have the interest to understand how the ILO estimates work, [you can read about it on this link](https://www.ilo.org/ilostat-files/Documents/TEM.pdf#:~:text=The%20ILO%20modelled%20estimates%20generally%20rely%20on%20a,the%20expected%20error%20and%20variance%20of%20the%20estimation.).

We saw that some indicators started to be recorded later than others. We can practice some aggregate functions to find the number of records, the first and the last year of record available for each indicator. In this query, we will be dropping the indicators that have ‘national estimate’ on their names.

In [ ]:
unemployment_bra_query = """
                        SELECT s.SeriesCode, s.IndicatorName, 
                            COUNT(s.SeriesCode) AS num_records, MIN(i.Year) AS min_year, MAX(i.Year) AS max_year
                        FROM Series AS s
                        INNER JOIN Indicators AS i
                            ON i.IndicatorCode = s.SeriesCode
                        WHERE i.CountryCode = 'BRA' AND s.Topic LIKE '%Unemployment' 
                            AND s.IndicatorName NOT LIKE '%national estimate%'
                        GROUP BY s.SeriesCode
                        ORDER BY num_records DESC;
                        """
unemployment_bra = pd.read_sql(unemployment_bra_query, conn)
unemployment_bra

Unfortunately, three interesting indicators have only 6 records (‘Share of youth not in education, employment or training’). They will be dropped from the final list of indicators.

# <a id="4">Extracting and Manipulating the Data for Analysis</a>

Let’s make our final SQL query to obtain all the data related to the selected indicators. We can make small modifications to our previous query and use it as a common table expression (CTE). This query will be referred as ‘Final_Indicators’ and, from it, we retrieve the data.

In [ ]:
final_query = """
                        WITH Final_Indicators AS
                        (
                            SELECT i.IndicatorName, i.Year, i.Value
                            FROM indicators AS i
                            INNER JOIN Series AS s
                                ON i.IndicatorCode = s.SeriesCode
                            WHERE i.CountryCode = 'BRA' AND s.Topic LIKE '%Unemployment' 
                                AND s.IndicatorName NOT LIKE '%national estimate%'
                                AND s.IndicatorName NOT LIKE 'Share%'
                        )  
                        
                        SELECT IndicatorName, Year, Value
                        FROM Final_Indicators;
                        """

final = pd.read_sql(final_query, conn)
final

Using the ‘pd.pivot’ function, we can transform the dataframe extracted into something we are more used to work with. With ‘Year’ serving as index, each indicator will be converted to a column on the new dataframe.

In [ ]:
df = final.copy().pivot(index = 'Year', columns = 'IndicatorName', values='Value')
df

In [ ]:
df.info()

Let’s simplify those column names.

In [ ]:
new_names = ['Long-term', 'Primary education', 'Primary education - Female', 'Primary education - Male'
            , 'Secondary education', 'Secondary education - Female', 'Secondary education - Male'
            , 'Tertiary education', 'Tertiary education - Female', 'Tertiary education - Male'
            , 'Female', 'Male', 'Total', 'Youth Female', 'Youth Male', 'Youth Total']
df = df.set_axis(new_names, axis=1)
df.info()

Finally, the data is ready to be plotted and analyzed.

In [ ]:
#sns.set_style("darkgrid")

plt.figure(figsize=(14,5))

sns.lineplot(data = df, x = df.index, y = 'Total', marker = 'o')
plt.title('Total Unemployment - From 1991 up to 2014')
plt.ylim((0,15))
#plt.yticks(range(0,110,10))
plt.ylabel('% of unemployment')

plt.show()

In [ ]:
plt.figure(figsize=(14,5))

sns.lineplot(data = df, x = df.index, y = 'Total', marker = 'o', label = 'Total')
sns.lineplot(data = df, x = df.index, y = 'Male', marker = 'o', label = 'Male')
sns.lineplot(data = df, x = df.index, y = 'Female', marker = 'o', label = 'Female')

plt.title('Unemployment by Gender - From 1991 up to 2014')
plt.ylim((0,15))
#plt.yticks(range(0,110,10))
plt.ylabel('% of unemployment')
plt.legend()

plt.show()

In [ ]:
plt.figure(figsize=(14,5))

sns.lineplot(data = df, x = df.index, y = 'Total', marker = 'o', label = 'Total')
sns.lineplot(data = df, x = df.index, y = 'Youth Total', marker = 'o', label = 'Youth')

plt.title('Unemployment: Total vs Youth Population (Ages 15 to 24) - From 1991 up to 2014')
plt.ylim((0,30))
#plt.yticks(range(0,110,10))
plt.ylabel('% of unemployment')
plt.legend()

plt.show()

In [ ]:
plt.figure(figsize=(14,5))

sns.lineplot(data = df, x = df.index, y = 'Youth Total', marker = 'o', label = 'Total')
sns.lineplot(data = df, x = df.index, y = 'Youth Male', marker = 'o', label = 'Male')
sns.lineplot(data = df, x = df.index, y = 'Youth Female', marker = 'o', label = 'Female')

plt.title('Unemployment: Youth Population (Ages 15 to 24) by Gender - From 1991 up to 2014')
plt.ylim((0,30))
#plt.yticks(range(0,110,10))
plt.ylabel('% of unemployment')
plt.legend()

plt.show()

In [ ]:
plt.figure(figsize=(14,5))

sns.lineplot(data = df, x = df.index, y = 'Total', marker = 'o', label = 'Total')
sns.lineplot(data = df, x = df.index, y = 'Primary education', marker = 'o', label = 'Primary')
sns.lineplot(data = df, x = df.index, y = 'Secondary education', marker = 'o', label = 'Secondary')
sns.lineplot(data = df, x = df.index, y = 'Tertiary education', marker = 'o', label = 'Tertiary')

plt.title('Unemployment by Education Level - From 2001 up to 2013')
plt.ylim((0,65))
#plt.yticks(range(0,110,10))
plt.ylabel('% of unemployment')
plt.xlim((2001,2013))
plt.legend()

plt.show()

In [ ]:
plt.figure(figsize=(14,5))

sns.lineplot(data = df, x = df.index, y = 'Primary education', marker = 'o', label = 'Total')
sns.lineplot(data = df, x = df.index, y = 'Primary education - Male', marker = 'o', label = 'Male')
sns.lineplot(data = df, x = df.index, y = 'Primary education - Female', marker = 'o', label = 'Female')

plt.title('Unemployment on Primary Education by Gender - From 2001 up to 2013')
plt.ylim((0,65))
#plt.yticks(range(0,110,10))
plt.ylabel('% of unemployment')
plt.xlim((2001,2013))
plt.legend()

plt.show()

In [ ]:
plt.figure(figsize=(14,5))

sns.lineplot(data = df, x = df.index, y = 'Secondary education', marker = 'o', label = 'Total')
sns.lineplot(data = df, x = df.index, y = 'Secondary education - Male', marker = 'o', label = 'Male')
sns.lineplot(data = df, x = df.index, y = 'Secondary education - Female', marker = 'o', label = 'Female')

plt.title('Unemployment on Secondary Education by Gender - From 2001 up to 2013')
plt.ylim((0,65))
#plt.yticks(range(0,110,10))
plt.ylabel('% of unemployment')
plt.xlim((2001,2013))
plt.legend()

plt.show()

In [ ]:
plt.figure(figsize=(14,5))

sns.lineplot(data = df, x = df.index, y = 'Tertiary education', marker = 'o', label = 'Total')
sns.lineplot(data = df, x = df.index, y = 'Tertiary education - Male', marker = 'o', label = 'Male')
sns.lineplot(data = df, x = df.index, y = 'Tertiary education - Female', marker = 'o', label = 'Female')

plt.title('Unemployment on Tertiary Education by Gender - From 2001 up to 2013')
plt.ylim((0,15))
#plt.yticks(range(0,110,10))
plt.ylabel('% of unemployment')
plt.xlim((2001,2013))
plt.legend()

plt.show()

# <a id="5">References</a>

- https://www.sqlite.org/schematab.html
- https://www.ilo.org/ilostat-files/Documents/TEM.pdf#:~:text=The%20ILO%20modelled%20estimates%20generally%20rely%20on%20a,the%20expected%20error%20and%20variance%20of%20the%20estimation.

## <center> If you find this notebook useful, support with an upvote! <center>